# 在束 gamma

In [1]:
%jsroot on
TFile *ipf = new TFile("../SeperateData/decay47_128.root");
//TFile *ipf = new TFile("/data/d2/zhli/hepfarm/ur12ana/test/srootall/bg47_128_all.root");
TTree *tree = (TTree*)ipf->Get("tree");
TCanvas *c1 = new TCanvas;

In [2]:
tree->Draw("ibeta>>(10,0,10)");
c1->Draw();

In [3]:
tree->Draw("bge:bgt>>(1000,-60000,5000,1500,0,1500)","ibeta==1","colz");
c1->SetLogz();
c1->Draw();
gROOT->Macro("bdelaycut.C");
gROOT->Macro("bdelaycut_bg.C");
c1->Draw();

In [4]:
//%jsroot off
tree->Draw("ge>>h(3000,0,3000)","gt<-150 && gt>-10000");
c1->Draw();

In [5]:
tree->Draw("bge>>h(3000,0,3000)","delaycut && ibeta<=1");
tree->Draw("bge>>hbg(3000,0,3000)","delaycutbg && ibeta<=1");
TH1D *h = (TH1D*)gROOT->FindObject("h");
TH1D *hbg = (TH1D*)gROOT->FindObject("hbg");
h->Add(hbg,-1);
h->Draw();
c1->Draw();

In [6]:
vector<double> e;            //e[0]为目标峰，其他为周围峰
double k,b;                  //线性本底参数
double fit_left,fit_right;   //联合拟合范围
double area_left,area_right; //峰面积计算范围

In [7]:
void area()
{
    c1->Clear();
    h->GetXaxis()->SetRangeUser(0,3500);        //返回全区间，在多次进行拟合时，必须要有这个操作，否则指定的拟合区间将存在问题

    //设置拟合函数：线性本底 + npeaks个高斯函数
    int npeaks = e.size();
    TString sf = "pol1(0)";
    for (int i=0; i<npeaks; i++)
        sf += Form("+gaus(%d)",3*i+2);
    TF1* f = new TF1("f", sf, fit_left, fit_right);
    f->FixParameter(0, b);                         //手动设置线性本底
    f->FixParameter(1, k);
    for (int i=0; i<npeaks; i++){
        f->SetParameter(3*i+2, 100);               //峰高
        f->SetParLimits(3*i+2, 1, 1e10);
        f->SetParameter(3*i+3, e[i]);              //峰中心
        f->SetParLimits(3*i+3, e[i]-1, e[i]+1);
        f->SetParameter(3*i+4, 1);                 //峰宽sigma
        f->SetParLimits(3*i+4, 0.5, 2);
    }
    
    TFitResultPtr fh = h->Fit(f, "QRS");
    f->SetLineColor(kRed);
    f->Draw("same");

    //单独提出线性本底
    TF1 *f1 = new TF1("f1", "pol1", 0, 3500);
    f1->SetParameter(0, b);
    f1->SetParameter(1, k);
    f1->SetLineColor(kBlue);
    f1->Draw("same");
    
    //提出联合拟合结果中的各个高斯峰
    TF1 *fg[npeaks];
    for (int i=0; i<npeaks; i++){
        fg[i] = new TF1(Form("fg%d",i), "gaus");
        fg[i]->SetParameter(0, f->GetParameter(3*i+2));
        fg[i]->SetParameter(1, f->GetParameter(3*i+3));
        fg[i]->SetParameter(2, f->GetParameter(3*i+4));
    }

    //计算峰面积
    int bin1 = h->FindBin(area_left);
    int bin2 = h->FindBin(area_right);
    double nall = h->Integral(bin1, bin2);                     //包含本底的总计数

    double x1 = h->GetBinLowEdge(bin1);                        //积分下界
    double x2 = h->GetBinLowEdge(bin2)+h->GetBinWidth(bin2);   //积分上界，注意使用了与TH1完全相等的积分范围
    double nbg = f1->Integral(x1, x2)/h->GetBinWidth(bin2);    //线性本底贡献
    for (int i=1; i<npeaks; i++)
        nbg += fg[i]->Integral(x1, x2)/h->GetBinWidth(bin2);   //其他高斯峰贡献；fg[0]是目标峰，不算进本底中

    //输出结果
    for (int i=0; i<npeaks; i++)
        cout<<i<<" peak="<<fg[i]->GetParameter(1)<<" height="<<fg[i]->GetParameter(0)
        <<" sigma="<<fg[i]->GetParameter(2)<<" contribution="<<fg[i]->Integral(x1, x2)/h->GetBinWidth(bin2)<<'\n';
    cout<<"chi2/ndf="<<fh->Chi2()/fh->Ndf()<<" count="<<nall-nbg<<" delta_count="<<sqrt(nall+nbg)<<endl;
}

In [8]:
e = vector<double> {200};
k = -0.05; b = 25;                      //线性本底参数
fit_left = e[0]-10; fit_right = e[0]+10;       //联合拟合范围
area_left = e[0]-2; area_right = e[0]+2;     //峰面积计算范围
area();
h->GetXaxis()->SetRangeUser(e[0]-50,e[0]+50); //设置显示区间
c1->Draw();

0 peak=200.334 height=218.535 sigma=0.862348 contribution=470.309
chi2/ndf=1.87913 count=470.125 delta_count=24.8973


In [9]:
e = vector<double> {527};
k = -0.02; b = 11.5;                      //线性本底参数
fit_left = e[0]-10; fit_right = e[0]+10;       //联合拟合范围
area_left = e[0]-3; area_right = e[0]+3;     //峰面积计算范围
area();
h->GetXaxis()->SetRangeUser(e[0]-50,e[0]+50); //设置显示区间
c1->Draw();

0 peak=527.573 height=112.054 sigma=0.90421 contribution=253.944
chi2/ndf=2.62325 count=263.35 delta_count=16.6328


In [10]:
e = vector<double> {1325};
k = 0; b = 0;                      //线性本底参数
fit_left = e[0]-10; fit_right = e[0]+10;       //联合拟合范围
area_left = e[0]-3; area_right = e[0]+4;     //峰面积计算范围
area();
h->GetXaxis()->SetRangeUser(e[0]-50,e[0]+50); //设置显示区间
c1->Draw();

0 peak=1326 height=54.9061 sigma=1.22171 contribution=167.965
chi2/ndf=1.59863 count=175 delta_count=13.2288


In [11]:
ofstream ofs("spectra/128Ag_singles.txt");
for (int ibin=1; ibin<=2000; ibin++){
    ofs<<h->GetBinContent(ibin)<<'\n';
}
ofs.close();